In [1]:
# ************FINAL CODE************
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from imutils.video import VideoStream
import numpy as np
import imutils#custom package which makes it easier for OpenCV
import time#importing time as it gives us current day counted from initial day.
import cv2
import joblib as joblib
import os
import time
import sqlite3
qrCodeDetector = cv2.QRCodeDetector()

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:
conn = sqlite3.connect('Entries.db')
c = conn.cursor()#created cursor to work with table
c.execute("CREATE TABLE entries (first text, plate text, phone text, number_fine integer, is_visited integer, day integer)")

In [ ]:
#The code can be executed only after providing the correct acc_sid and auth_token which is provided by twilio
#Note!
#acc_sid and auth_token should not be shared with others.
from twilio.rest import Client
acc_sid = "ACe31eca31f87b346xxxxxxxxxxxx417cc5"#dummy number acc_sid
auth_token = "1e516637b70c522dxxxxxxxxxxxx29a738c"#dummy number auth_token

client = Client(acc_sid, auth_token)#creation of client

In [ ]:
def generate_fine(string):
    details=string.split()
    ph = details[2]#Phone number extracted from QR Code
#     print("Fine generated")
    with conn:
      c.execute("SELECT number_fine, is_visited FROM Entries WHERE phone=:phone", {'phone':ph})
      li = c.fetchall()[0]
      if(li[0]==0 and li[1]==0):#if both number_fine and is_visited are 0 
      #both zero means person is being scanned for first time.
        day_now = time.localtime(time.time())[7] #Extracting day from struct of time!
        c.execute("UPDATE Entries SET number_fine = :number_fine, is_visited = :is_visited, day = :day WHERE phone = :phone",{'number_fine':1, 'is_visited':1, 'day':day_now, 'phone':ph})
        #Also storing day in DB for future reference.
        print("First Warning")
        #Twilio warning message
        client.messages.create(
                                to = "+91"+ph,
                                from_ = "+12018977421",
                                body = "This is your first warning, as you were not wearing a mask today while driving"
                              )
      elif(li[0]!=0 and li[1]==0):#if number_fine is not 0 but is_visited is 0
      #This situation indicates that a person has been warned before but not warned today, hence pay the fine.
        day_now = time.localtime(time.time())[7]#Extrating Current Day and storing in db for future.
        c.execute("UPDATE Entries SET is_visited = :is_visited, day = :day WHERE phone = :phone",{'is_visited':1, 'day':day_now, 'phone':ph})
        print("You have to give fine")
        client.messages.create(
                                to = "+91"+ph,
                                from_ = "+12018977421",
                                body = "This is a message for you stating your fine, as you were not wearing a mask today while driving even after a final warning"
                              )
      else:
        #if number fine is not 0 and is_visited is also not 0
        #means person has been encountered before.
        c.execute("SELECT day FROM Entries WHERE phone=:phone", {'phone':ph})
        client_day = c.fetchall()[0][0]#Extracting the day when he was encountered last time.
        now_day = time.localtime(time.time())[7]#Extracting current day
        if(now_day <= client_day): #if current day is greater, MORE THAN 24 hrs of TIME!
          print("Already given fine for today!")#24 hrs not passed and he has already given fine.
        else:#24 hrs have passed and he has to give fine.
          c.execute("UPDATE Entries SET is_visited = :is_visited Where phone = :phone", {'is_visited':0, 'phone':ph})
          print("No Mask found again today (Final Warning)")
          client.messages.create(
                                to = "+91"+ph,
                                from_ = "+12018977421",
                                body = "This is your final warning, as you were not wearing a mask today while driving. You have to give fine after this."
                                )

In [ ]:
#LIVE Mask Detection System
def detect_and_predict_mask(frame, faceNet, maskNet):
	# grab the dimensions of the frame and then construct a blob
	# from it
	(h, w) = frame.shape[:2]
	blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300),
		(104.0, 177.0, 123.0))

	# pass the blob through the network and obtain the face detections
	faceNet.setInput(blob)
	detections = faceNet.forward()

	# initialize our list of faces, their corresponding locations,
	# and the list of predictions from our face mask network
	faces = []
	locs = []
	preds = []

	# loop over the detections
	for i in range(0, detections.shape[2]):
		# extract the confidence (i.e., probability) associated with
		# the detection
		confidence = detections[0, 0, i, 2]

		# filter out weak detections by ensuring the confidence is
		# greater than the minimum confidence
		if confidence >0.5:
			# compute the (x, y)-coordinates of the bounding box for
			# the object
			box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
			(startX, startY, endX, endY) = box.astype("int")

			# ensure the bounding boxes fall within the dimensions of
			# the frame
			(startX, startY) = (max(0, startX), max(0, startY))
			(endX, endY) = (min(w - 1, endX), min(h - 1, endY))

			# extract the face ROI, convert it from BGR to RGB channel
			# ordering, resize it to 224x224, and preprocess it
			face = frame[startY:endY, startX:endX]
			face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
			face = cv2.resize(face, (64, 64))
			face = img_to_array(face)
			face = np.expand_dims(face, axis=0)

			# add the face and bounding boxes to their respective
			# lists
			faces.append(face)
			locs.append((startX, startY, endX, endY))
            
	if len(faces)==1:
		preds = maskNet.predict(faces)
	if len(faces)>1:
		locs=[]

	# return a 2-tuple of the face locations and their corresponding
	# locations
	return (locs, preds)

# load our serialized face detector model from disk
print("[INFO] loading face detector model...")
faceNet = cv2.dnn.readNet("deploy.prototxt", "res10_300x300_ssd_iter_140000.caffemodel")

# load the face mask detector model from disk
print("[INFO] loading face mask detector model...")
maskNet = joblib.load('loaded_model_updated_dataset_3.pkl')

# initialize the video stream and allow the camera sensor to warm up
print("[INFO] starting video stream...")
vs = VideoStream(src=0).start()
time.sleep(2.0)

# loop over the frames from the video stream
check=0 #If No Mask is detected in the face value of check will be changed to 1 and fine will be generated.
photo_no=1
t_end=time.time()
while True:
	# grab the frame from the threaded video stream and resize it
	# to have a maximum width of 400 pixels
	frame = vs.read()
	# frame = imutils.resize(frame, width=400)

	if check==1:
		#Camera moves downward along Y-axis to detect qrcode for the details 
		if time.time() < t_end:
			decodedText, points, _ = qrCodeDetector.detectAndDecode(frame) 
			if len(decodedText)>0:
				generate_fine(decodedText)
				check=0
		else:
			#This part is executed when a QR Code is not found and hence the image is stored for charging fine manually.
			cv2.imwrite("Image_"+str(photo_no)+".jpg",frame)
			photo_no+=1      
			check=0
		cv2.imshow("Frame", frame)
		key = cv2.waitKey(1) & 0xFF
		if key == ord("q"):
			break
		continue
        
	# detect faces in the frame and determine if they are wearing a
	# face mask or not
	(locs, preds) = detect_and_predict_mask(frame, faceNet, maskNet)

	# loop over the detected face locations and their corresponding
	# locations
	for (box, pred) in zip(locs, preds):
		# unpack the bounding box and predictions
		(startX, startY, endX, endY) = box
		withoutMask = pred[0]

		# determine the class label and color we'll use to draw
		# the bounding box and text
		label = "Mask" if (1-withoutMask) > withoutMask else "No Mask"
		color = (0, 255, 0) if label == "Mask" else (0, 0, 255)
		check=0
		if label=="No Mask":
			check=1
		# include the probability in the label
		label = "{}: {:.2f}%".format(label, max((1-withoutMask), withoutMask) * 100)

		# display the label and bounding box rectangle on the output
		# frame
		cv2.putText(frame, label, (startX, startY - 10),
			cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
		cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)
		if check==1:
			t_end=time.time()+10
	# show the output frame
	cv2.imshow("Frame", frame)
	key = cv2.waitKey(1) & 0xFF

	# if the `q` key was pressed, break from the loop
	if key == ord("q"):
		break
	if check==1:
        #If no Mask is found a time break is taken so that it could be visible in LIVE system or else it vanishes within a flash of moment.
        #And now,the system checks for QR Code to generate fine.
		time.sleep(1)

# do a bit of cleanup
vs.stop()
cv2.destroyAllWindows()